Модуль для расчета временных файлов, для быстроты построения моделей, файлов для heroku

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

DIR = '/content/drive/My Drive/Colab Notebooks/Data_Sceince/Module_5/recommendationsv4/'
DIR_TEST = '/content/drive/My Drive/Colab Notebooks/Data_Sceince/Module_5/TEST/'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
pip install lightfm

In [ ]:
import pandas as pd
import sklearn
import pickle
import json

import scipy.sparse as sparse

from lightfm import LightFM
from sklearn.model_selection import train_test_split

from lightfm.data import Dataset

In [ ]:
#Читаем файлы
data = pd.read_csv(DIR + 'train.csv',low_memory=False )
test = pd.read_csv(DIR +'test.csv',low_memory=False )

# Определяем выдающиеся значения, вероятно, спам, исключим влияние этих оценок
def get_spam(field):
  med = data[field].value_counts().describe()['50%']
  std = data[field].value_counts().describe()['std']
  counts = pd.DataFrame(data[field].value_counts())
  spam = list(counts[counts[field] > med + 3 * std].index)
  return spam

item_spam = get_spam('itemid')
user_spam = get_spam('userid')
data = data[~((data['itemid'].isin(item_spam)) & (data['userid'].isin(user_spam)))]


def cut_meta_json():
  #Запишем урезанный json , только те asin , которые есть в трейне и тесте, из set_asin
    set_asin = set(data['asin']) | set(test['asin'])
    list_meta = []
    with open(DIR + 'meta_Grocery_and_Gourmet_Food.json') as file:
        for line in file.readlines(): 
            lines = json.loads(line)
            if lines.get('asin',0) in set_asin:
# нам интересен этот товар
                list_meta.append(line)

    with open(DIR +"meta_Grocery_and_Gourmet_Food_small.json", "w") as file:
        file.writelines("%s" % line for line in list_meta)
        file.close()


cut_meta_json()


#Формируем  mapping для всех пользователей и товаров(train + test)

dataset = Dataset() #item_identity_features=False)
dataset.fit((x for x in pd.concat([data, test])['userid']),
            (x for x in pd.concat([data, test])['itemid']))

# Вытаскиваем словари для преобразований
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()

#Сохраняем словари
with open(DIR + 'user_id_map_base0.pkl', 'wb') as output:
                pickle.dump(user_id_map, output)

with open(DIR + 'item_id_map_base0.pkl', 'wb') as output:
                pickle.dump(item_id_map, output)

#Готовим основную матрицу на всем датасете
(interactions, weights) = dataset.build_interactions(((x,y,z) 
                for x,y, z in zip(data.userid.values,data.itemid.values, data.rating.values)))

NUM_THREADS = 8 #число потоков
RANDOM_SEED = 32
#Гиперпараметры
NUM_EPOCHS, LEARNING_RATE, NUM_COMPONENTS, ALPHA =  [108,0.9307615113531499,156,1.117646374973343e-06] # без фич

# Модель
model_light = LightFM(learning_rate=LEARNING_RATE, loss='logistic', no_components=NUM_COMPONENTS,
                      random_state=32,
                      user_alpha=ALPHA,
                      item_alpha=ALPHA)

model_light = model_light.fit(weights, epochs=NUM_EPOCHS, num_threads=NUM_THREADS)
#Сохраняем модель  для heroku
with open(DIR + 'myfile_base0.pkl', 'wb') as output:
                pickle.dump(model_light, output)

print('end')

end


In [ ]:
preds_light = model_light.predict(data.userid.apply(lambda x: user_id_map[x]).values,
                              data.itemid.apply(lambda x: item_id_map[x]).values )
sklearn.metrics.roc_auc_score(data.rating,preds_light)

0.9897994436028028

In [ ]:
#Разбиваем наш датасет на обучающую и тестовую выборки
train_data, test_data = train_test_split(data,random_state=32, shuffle=True)

In [ ]:
#Готовим основную матрицу на всем train
(interactions, weights) = dataset.build_interactions(((x,y,z) 
                for x,y, z in zip(train_data.userid.values,train_data.itemid.values, train_data.rating.values)))

NUM_THREADS = 8 #число потоков
RANDOM_SEED = 32
#Гиперпараметры
NUM_EPOCHS, LEARNING_RATE, NUM_COMPONENTS, ALPHA =  [108,0.9307615113531499,156,1.117646374973343e-06] # без фич

# Модель
model_light = LightFM(learning_rate=LEARNING_RATE, loss='logistic', no_components=NUM_COMPONENTS,
                      random_state=32,
                      user_alpha=ALPHA,
                      item_alpha=ALPHA)

model_light = model_light.fit(weights, epochs=NUM_EPOCHS, num_threads=NUM_THREADS)

print('end')

end


In [ ]:
#Проверяем на тесте
preds_light = model_light.predict(test_data.userid.apply(lambda x: user_id_map[x]).values,
                              test_data.itemid.apply(lambda x: item_id_map[x]).values )

In [ ]:
#Прикинем что получится
sklearn.metrics.roc_auc_score(test_data.rating,preds_light)
# 0.7556278146479729  без очистки данных
#0.7544194734924881 со очисткой

0.7544194734924881

In [ ]:
# Определяем выдающиеся значения, вероятно, спам, исключим влияние этих оценок
def get_spam(field):
  med = data[field].value_counts().describe()['50%']
  std = data[field].value_counts().describe()['std']
  # med = data[field].value_counts().describe()['75%']
  # std = (data[field].value_counts().describe()['75%'] - data[field].value_counts().describe()['25%'])/2
  counts = pd.DataFrame(data[field].value_counts())
  spam = list(counts[counts[field] > med + 2 * std].index)
  return spam

data = pd.read_csv(DIR + 'train.csv',low_memory=False )

item_spam = get_spam('itemid')
user_spam = get_spam('userid')
data = data[~((data['itemid'].isin(item_spam)) & (data['userid'].isin(user_spam)))]

# Упорядочим наши данные по верифицированности ответа и времени
data.sort_values(by = ['verified', 'unixReviewTime'], ascending= True, inplace = True)

#Уберем  дубли, оставим только последний отзыв пользователя на один товар
data.drop_duplicates(subset =['userid','unixReviewTime','itemid'], keep = 'last',inplace = True)
data = data[['userid','reviewerName','itemid']]
#Данные по пользователям для heroku
data.to_csv(DIR + 'data_clean.csv')


In [ ]:
#Словарь соответствия asin и itemid
data_all = pd.concat([data, test])
data_all = data_all[['asin','itemid']]
data_all.drop_duplicates(subset =['itemid'], keep = 'last',inplace = True)
mapper = dict(zip(data_all.asin, data_all.itemid))

def get_by_key(field, dict_from, dict_to):
   if field in dict_from.keys():
       dict_to[mapper.get(dict_from.get('asin'))] = dict_from.get(field)[:200]    

with open(DIR + 'meta_Grocery_and_Gourmet_Food_small.json') as file:
     d_title = {}
     for line in file.readlines(): 
         lines = json.loads(line) 
         get_by_key('title', lines, d_title)

#Данные по товарам для heroku
with open(DIR + 'item_title_map_base0.pkl', 'wb') as output:
                pickle.dump(d_title, output)           